In [7]:
# %%
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

# %%
# Load pre-trained models and data
with open('./models/movies.pkl', 'rb') as f:
    movies_df = pickle.load(f)
with open('./models/ratings.pkl', 'rb') as f:
    ratings_df = pickle.load(f)
with open('./models/nmf_model.pkl', 'rb') as f:
    nmf_model = pickle.load(f)
with open('./models/tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)
with open('./models/user_item_matrix.pkl', 'rb') as f:
    user_item_matrix = pickle.load(f)

# %%
# Ask user for ID
user_id = int(input("Enter your user ID: "))

# Recommend initial random movies
sample_movies = movies_df[['movie_id', 'title']].sample(5, random_state=42)
print("\n⭐ Please rate the following movies (1 to 5 stars):\n")
feedback_list = []

for _, row in sample_movies.iterrows():
    while True:
        try:
            rating = int(input(f"Rate '{row['title']}' (1-5): "))
            if 1 <= rating <= 5:
                feedback_list.append({
                    'user_id': user_id,
                    'movie_id': row['movie_id'],
                    'title': row['title'],
                    'rating': rating
                })
                break
            else:
                print("⚠️ Please enter a number between 1 and 5.")
        except ValueError:
            print("⚠️ Invalid input. Enter a number between 1 and 5.")

# %%
# Create feedback DataFrame
feedback_df = pd.DataFrame(feedback_list)

# Identify liked movies (rated 4 or 5)
liked_movie_ids = feedback_df[feedback_df['rating'] >= 4]['movie_id'].tolist()

# Recommend similar movies
if liked_movie_ids:
    liked_indices = movies_df[movies_df['movie_id'].isin(liked_movie_ids)].index.tolist()

    cosine_scores = cosine_similarity(
        tfidf_matrix[liked_indices],
        tfidf_matrix
    ).mean(axis=0)

    movies_df['similarity'] = cosine_scores
    recommendations = movies_df[~movies_df['movie_id'].isin(liked_movie_ids)]
    top_recs = recommendations[['movie_id', 'title', 'similarity']].nlargest(5, 'similarity')

    print("\n🎯 Recommended for you based on your feedback:\n")
    display(top_recs[['title', 'similarity']])

    print("\n🎬 Pick a movie from the above to enjoy watching!")

else:
    print("No highly rated movies provided. Cannot generate personalized recommendations.")

# %%
# Save feedback to ratings.csv
try:
    existing_ratings = pd.read_csv("ratings.csv")
    updated_ratings = pd.concat([existing_ratings, feedback_df[['user_id', 'movie_id', 'rating']]], ignore_index=True)
except FileNotFoundError:
    updated_ratings = feedback_df[['user_id', 'movie_id', 'rating']]

updated_ratings.to_csv("ratings.csv", index=False)
print("\n✅ Your ratings have been saved for future recommendations in 'ratings.csv'.")



⭐ Please rate the following movies (1 to 5 stars):


🎯 Recommended for you based on your feedback:



,title,similarity
171,"Empire Strikes Back, The (1980)",0.601380
630,"Crying Game, The (1992)",0.507093
5,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,0.500000
8,Dead Man Walking (1995),0.500000
14,Mr. Holland's Opus (1995),0.500000



🎬 Pick a movie from the above to enjoy watching!

✅ Your ratings have been saved for future recommendations in 'ratings.csv'.
